In [1]:
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv("coursera_sessions_train.txt", sep=';', header=None)
data[:15]

,0,1
0,"0,1,2,3,4,5",NaN
1,"9,10,11,9,11,12,9,11",NaN
2,"16,17,18,19,20,21",NaN
3,"24,25,26,27,24",NaN
4,"34,35,36,34,37,35,36,37,38,39,38,39",NaN
5,42,NaN
6,"47,48,49",NaN
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63"
8,"71,72,73,74",NaN
9,"76,77,78",NaN


In [5]:
data.columns = ['view', 'buy']
data.head()

,view,buy
0,"0,1,2,3,4,5",NaN
1,"9,10,11,9,11,12,9,11",NaN
2,"16,17,18,19,20,21",NaN
3,"24,25,26,27,24",NaN
4,"34,35,36,34,37,35,36,37,38,39,38,39",NaN


In [6]:
x = "lksjdlkfj"

In [27]:
view_freq = {}
for row in data.view.values:
    l = [x.strip() for x in row.split(',')]
    for i in l:
        if view_freq.get(i) == None:
            view_freq[i] = 1
        else:
            view_freq[i] += 1
            
keys = sorted(view_freq, key=view_freq.get, reverse=True)
for k in keys[:10]:
    print k, ' ', view_freq[k]
    

73   677
158   641
204   396
262   387
162   318
7   312
137   306
1185   284
6   283
170   280


In [28]:
buy_freq = {}
for row in data.buy.dropna().values:
    l = [x.strip() for x in str(row).split(',')]
    for i in l:
        if buy_freq.get(i) == None:
            buy_freq[i] = 1
        else:
            buy_freq[i] += 1
            
keys = sorted(buy_freq, key=buy_freq.get, reverse=True)
for k in keys[:10]:
    print k, ' ', buy_freq[k]

158   14
204   12
3324   11
73   11
977   10
5569   10
3149   10
1181   9
162   8
1852   7


In [13]:
def drop_dup(s):
    l = [x.strip() for x in str(s).split(',')]
    res = []
    for i in l:
        if i not in res:
            res.append(i)
            
    return ",".join(res)

In [14]:
filtered = data.dropna()
filtered.view = filtered.view.apply(drop_dup)
filtered.head()

C:\Users\yuranich\Anaconda2\lib\site-packages\pandas\core\generic.py:2698: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,view,buy
7,"59,60,61,62,63,64,65,66,67,68","67,60,63"
10,"84,85,86,87,88,89,90,91,92,93",86
19,"138,198,199,127",199
30,"303,304,305,306,307,308,309,310,311,312",303
33,"352,353",352


In [67]:
def getSorted(s, d):
    l = [x.strip() for x in s.split(',')]
    tuples = []
    for i in l:
        if d.get(i) == None:
            tuples.append((0, i))
        else:
            tuples.append((d[i], i))
    ht = lambda x, y : y[0] - x[0] if x[0] != y[0] else l.index(x[1]) - l.index(y[1])
    return sorted(tuples, cmp=ht)

def suggest5view(s):
    rec = getSorted(s, view_freq)
    if len(rec) > 5:
        return map(lambda x : x[1], rec[:5])
    else:
        return map(lambda x : x[1], rec)

def suggest5buy(s):
    rec = getSorted(s, buy_freq)
    if len(rec) > 5:
        return map(lambda x : x[1], rec[:5])
    else:
        return map(lambda x : x[1], rec)

In [52]:
suggest1view("59,60,61,62,63,64,65,66,67,68")

'63'

In [93]:
ap1 = []
ap5 = []
ar1 = []
ar5 = []
ap1buy = []
ap5buy = []
ar1buy = []
ar5buy = []
for row in filtered.iterrows():
    res = suggest5view(row[1]['view'])
    one = res[0]
    buy_l = [x.strip() for x in row[1]['buy'].split(',')]
    p1 = 1 if one in buy_l else 0
    r1 = (float(p1)) / len(buy_l)
    nom = sum([1 if x in buy_l else 0 for x in res])
    p5 = float(nom) / 5
    r5 = float(nom) / len(buy_l)
    ap1.append(p1)
    ap5.append(p5)
    ar1.append(r1)
    ar5.append(r5)
    
    res_buy = suggest5buy(row[1]['view'])
    one_buy = res_buy[0]
    p1_buy = 1 if one_buy in buy_l else 0
    r1_buy = (float(p1_buy)) / len(buy_l)
    nom = sum([1 if x in buy_l else 0 for x in res_buy])
    p5_buy = float(nom) / 5
    r5_buy = float(nom) / len(buy_l)
    ap1buy.append(p1_buy)
    ap5buy.append(p5_buy)
    ar1buy.append(r1_buy)
    ar5buy.append(r5_buy)

In [82]:
def writeArr(arr, filename):
    with open(filename, "w") as f:
        res = " ".join(map(str, arr))
        f.write(res)

In [94]:
arg = [sum(ar1) / float(len(ar1)), sum(ap1) / float(len(ap1)), sum(ar5) / float(len(ar5)), sum(ap5) / float(len(ap5))]
writeArr(arg, "train_res_view.txt")

In [95]:
arg = [sum(ar1buy) / float(len(ar1buy)), sum(ap1buy) / float(len(ap1buy)), sum(ar5buy) / float(len(ar5buy)), sum(ap5buy) / float(len(ap5buy))]
writeArr(arg, "train_res_buy.txt")

In [85]:
test = pd.read_csv("coursera_sessions_test.txt", sep=';', header=None)
test[:15]

,0,1
0,"6,7,8",NaN
1,"13,14,15",NaN
2,"22,23",NaN
3,"28,29,30,31,32,33",NaN
4,"40,41",NaN
5,"43,44,43,45,43,45,43,46",NaN
6,"50,51,47,52,49,53,54,55,56,57,58",NaN
7,"63,68,69,70,66,61,59,61,66,68","66,63"
8,75,NaN
9,"79,80,81,82,83",NaN


In [86]:
test.columns = ['view', 'buy']
test_filt = test.dropna()
test_filt.view = test_filt.view.apply(drop_dup)
test_filt.head()

,view,buy
7,"63,68,69,70,66,61,59","66,63"
14,"158,159,160,161,162",162
19,"200,201,202,203,204","201,205"
34,"371,372","371,373"
40,422,422


In [92]:
ap1 = []
ap5 = []
ar1 = []
ar5 = []
ap1buy = []
ap5buy = []
ar1buy = []
ar5buy = []
for row in test_filt.iterrows():
    res = suggest5view(row[1]['view'])
    one = res[0]
    buy_l = [x.strip() for x in row[1]['buy'].split(',')]
    p1 = 1 if one in buy_l else 0
    r1 = (float(p1)) / len(buy_l)
    nom = sum([1 if x in buy_l else 0 for x in res])
    p5 = float(nom) / 5
    r5 = float(nom) / len(buy_l)
    ap1.append(p1)
    ap5.append(p5)
    ar1.append(r1)
    ar5.append(r5)
    
    res_buy = suggest5buy(row[1]['view'])
    one_buy = res_buy[0]
    p1_buy = 1 if one_buy in buy_l else 0
    r1_buy = (float(p1_buy)) / len(buy_l)
    nom = sum([1 if x in buy_l else 0 for x in res_buy])
    p5_buy = float(nom) / 5
    r5_buy = float(nom) / len(buy_l)
    ap1buy.append(p1_buy)
    ap5buy.append(p5_buy)
    ar1buy.append(r1_buy)
    ar5buy.append(r5_buy)
    
arg = [sum(ar1) / float(len(ar1)), sum(ap1) / float(len(ap1)), sum(ar5) / float(len(ar5)), sum(ap5) / float(len(ap5))]
writeArr(arg, "test_res_view.txt")

arg = [sum(ar1buy) / float(len(ar1buy)), sum(ap1buy) / float(len(ap1buy)), sum(ar5buy) / float(len(ar5buy)), sum(ap5buy) / float(len(ap5buy))]
writeArr(arg, "test_res_buy.txt")

In [91]:
check = {}
for row in data.buy.dropna()[:50].values:
    l = [x.strip() for x in str(row).split(',')]
    print l
    for i in l:
        if check.get(i) == None:
            check[i] = 1
        else:
            check[i] += 1
            
keys = sorted(check, key=check.get, reverse=True)
for k in keys[:10]:
    print k, ' ', check[k]

['67', '60', '63']
['86']
['199']
['303']
['352']
['519']
['603', '604', '602', '599', '605', '606', '600']
['690', '688']
['851']
['879']
['1118']
['1545']
['1727']
['99']
['1907']
['1959']
['1998']
['2013']
['2019']
['2462']
['2520']
['2543']
['1526']
['2764']
['2857', '2853', '2852']
['2920']
['2930']
['3033', '3026', '3032', '3031']
['3102', '3100', '1260']
['3204', '3207', '3217']
['3267']
['3443']
['3529']
['3584', '3564', '3612', '3623', '3645', '3561']
['1662']
['3741']
['3979']
['4216']
['4246']
['4298']
['3066']
['4512']
['4750']
['4815']
['5058']
['5066']
['5134', '5135']
['5183', '970']
['5209']
['805']
605   1
604   1
3612   1
606   1
199   1
600   1
603   1
602   1
599   1
2920   1
